In [203]:
# n.1    Importamos librerias pertinentes

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [204]:
# n.2    Importamos el csv total_df

path = 'C:\\Users\\User\Desktop\\Proyecto_MLOps_SoyHenry\\files\\total_df.csv'

total_df = pd.read_csv(path)

In [205]:
# n.3     Creamos una columna llamada combined feat con la columna de title y listed_in para generar una matrix de similitud
#         entre las peliculas

def combine_features(data):
    features = []
    for i in range(0,total_df.shape[0]):
        features.append(data['title'][i] + ' ' +data['listed_in'][i])
    
    return features

total_df['combined_feat'] = combine_features(total_df)

In [207]:
# n.4      Usamos CountVectorizer de scikit-learn para crear vectores de las strings de la columna combined feat
#          para contar cuantas veces se repiten ciertas palabras.

cm = CountVectorizer().fit_transform(total_df['combined_feat'])

In [208]:
# n.5         Creamos una matrix de similitud entre las peliculas a partir del Count Vectorizer del paso anterior.

cs = cosine_similarity(cm)

In [209]:
# n.6         Creamos una funcion en la que insertamos el nombre de la peliculas y nos devuelve las 5 peliculas más recomendadas
#             en orden descendente.

def recommender_system(movie):
    movie_id = total_df[total_df['title']==movie].iloc[0][0]
    scores  = list(enumerate(cs[movie_id]))
    sorted_scores = sorted(scores,key = lambda x:x[1],reverse=True)
    return [total_df.iloc[i[0]]['title'] for i in sorted_scores[1:6]]

In [223]:
# Testeamos el sistema

recommender_system('zoom')

['beethoven', 'charming', 'leap!', 'rango', 'paranorman']

In [210]:
# Eliminamos la columna combined feat.

total_df.drop('combined_feat',axis=1,inplace=True)

In [211]:
# n.7       Aplicamos la funcion recommender_system a todo el DataFrame para poder insertar las recomendaciones
#           en el mismo DataFrame de las consultas y gastar menos memoria. 

total_df['recommender'] = total_df['title'].apply(recommender_system)

In [214]:
# n.9       Convertimos el DataFrame en un archivo csv para la creacion de la API.

df_fastapi = total_df.to_csv('df_fastapi.csv')